In [1]:
import sys
import tes.simulation.data as sim
import tes.simulation.plot as simplt
import numpy as np
import matplotlib.pyplot as plt
from tes.simulation.filesets import measurement_unit_TB as fileset

data = sim.Data(fileset, 1, 'teslib','measurement_unit_TB')
data.event_stream

array([], 
      dtype=[('data', '>u8'), ('last', '?')])

In [ ]:
sim.EventStream.Flags(es.bytestream[4:6])

In [ ]:
trace_header_dt = np.dtype(
    ([
            ('size',np.uint16),('tflags',np.uint8,(2,)),('dflags',np.uint8,(2,)),
            ('time',np.uint16),('offset',np.uint16),('p_thresh',np.uint16),
            ('s_thresh',np.uint16),('length',np.uint16),('resvd',np.uint16,(2,)),
            ('area',np.uint32)
     ])
)
trace_peak_dt = np.dtype(
    ([
            ('min',np.uint16),('height',np.uint16),('max',np.uint16),('time',np.uint16)
            
    ])
)
trace_dt = np.dtype(
    [('header',trace_header_dt),('peaks',trace_peak_dt,(1,)),('data',np.uint16,(600,))]
)

h=es.bytestream[0:24].view(trace_header_dt)
max_peaks=np.bitwise_and(h['tflags'][0][1],0x0F)
p=es.bytestream[24:24+(max_peaks+1)*8].view(trace_peak_dt)
t=es.bytestream[24+(max_peaks+1)*8:].view(np.int16)
t.shape=(-1,2)
h

In [ ]:
data.pulse_stop[0][0]

In [ ]:
%matplotlib notebook
starts=data.peak_start
stops=data.peak

eventstream_traces = []
data_traces = []

s=0
for peak in p:
    l=peak[2]-peak[0]+1
    eventstream_traces.append(t[s:s+l])
    
    plt.step(np.arange(peak[0],peak[2]+1),t[s:s+l,0]/2,'b')
    plt.step(np.arange(peak[0],peak[2]+1),t[s:s+l,1]/2,'r')
    s=l

plt.plot(data.trace['filtered'][data.event_start[0][0]:data.pulse_stop[0][0]]/2,':b')
plt.plot(data.trace['slope'][data.event_start[0][0]:data.pulse_stop[0][0]]/2,':r')

In [ ]:
eventstream_traces[0]

In [ ]:
%matplotlib notebook
plt.step(np.arange(len(t[:,0])),t[:,0]/2)
plt.step(np.arange(len(t[:,1])),t[:,1]/2,'r')

In [ ]:
d

In [ ]:
data.event_stream['last'].nonzero()

In [ ]:
peak_dt = np.dtype(
    ([ ('height', np.int16), ('minima', np.int16), ('flags', np.uint16), ('time',np.uint16)])
)
events = data.event_stream['data'].view(peak_dt)

In [ ]:
pulse_peak_dt = np.dtype(
    ([('height',np.uint16),('minima',np.uint16),('rise_time',np.uint16),('time',np.uint16)])
)
pulse_dt = np.dtype(
    ([('size',np.uint16),('length',np.uint16),('flags',np.uint16),('time',np.uint16),
     ('area',np.int32),('pulse_threshold',np.uint16),('slope_threshold',np.uint16),
     ('peak',pulse_peak_dt)])
)

In [ ]:
bytestream=np.copy(data.event_stream['data']).view(np.uint8)
bytestream

In [ ]:
events=bytestream.view(pulse_dt)
events

In [ ]:
events

In [ ]:
flags=sim.DetectionFlags(events['flags'][0])
flags


In [ ]:
bs=np.copy(data.event_stream['data']).view(np.uint8)

In [ ]:
event_size=bs[0:2].view(np.uint16)[0]
event_size
pulse_peaks=event_size-2
pulse_peak_dt = np.dtype(
    ([('height',np.uint16),('minima',np.uint16),('rise_time',np.uint16),('time',np.uint16)])
)
pulse_dt = np.dtype(
    ([('size',np.uint16),('length',np.uint16),('flags',np.uint8,(2,)),('time',np.uint16),
     ('area',np.int32),('pulse_threshold',np.uint16),('slope_threshold',np.uint16),
     ('peaks',pulse_peak_dt,(pulse_peaks))])
)

In [ ]:
bs.view(pulse_dt)['flags']

In [ ]:
es=sim.EventStream(data.event_stream)

In [ ]:
es.bytestream


In [ ]:
type_flags = np.bitwise_and(events['flags'],0x000f)
peak_num = np.right_shift(np.bitwise_and(events['flags'],0x00f0),4)
channel = np.right_shift(np.bitwise_and(events['flags'],0x0f00),8)
event_type = np.right_shift(np.bitwise_and(events['flags'],0x3000),12)

event_type


In [ ]:
%matplotlib notebook
fig = simplt.plot_pulse(8,data,pre=500,length=3000)


In [ ]:
import pickle

# An arbitrary collection of objects supported by pickle.

pickler = pickle.Pickler(open('short.pickle', 'wb'),pickle.HIGHEST_PROTOCOL)
pickler.dump(data)


In [ ]:
import pickle
unpickler = pickle.Unpickler(open('short.pickle', 'rb'))
data2=unpickler.load()

In [ ]:
data2.trace['slope']

In [ ]:
reg2

In [ ]:
axs = fig.get_axes()
pax=axs[0].parasites
pax[0]._label
f_ins_ax=axs[1]
s_ins_ax=f_ins_ax.parasites[0]

In [ ]:
es=sim.Data.fromfile('eventstream',np.int32,'teslib','measurement_unit_TB')

In [ ]:
es

In [ ]:
data.peak[-1]

In [ ]:
s.peak

In [ ]:
isinstance(r,sim.DataRegion)

In [ ]:
r.peak

In [ ]:
i

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

plt.plot(data.trace['filtered'])

In [ ]:
ax=plt.gca()
t=ax.transData
t.transform([[0.1,0.2],[1,3]])

In [ ]:
x=np.array([0,1,2,3])
y=np.array([3,4,5,6])

xy=np.empty(shape=(4,2))

In [ ]:
xy[:,0]=x
xy[:,1]=y
t.transform(xy)

In [ ]:
# run to import isim data files

import numpy as np
import os
import matplotlib.pyplot as plt
import math
from enum import Enum

plt.rcParams["figure.figsize"]=[800./80,600./80]

repo='c:\\TES_project\\fpga_ise\\'
project='teslib'
testbench='measurement_unit_TB'
datafile='traces'
path=repo+project+'\\PlanAhead\\'+project+'.sim\\'+testbench+'\\'

traces_dt = np.dtype([('input', np.int32),('raw',np.int32),('filtered',np.int32),('slope',np.int32)])
traces=np.fromfile(path+'traces',traces_dt)
trigger=np.fromfile(path+'triggers',np.int32)
settings=np.fromfile(path+'settings',np.int32)
cfd_low=np.fromfile(path+'cfdlow',np.int32)
cfd_high=np.fromfile(path+'cfdhigh',np.int32)
peaks=np.fromfile(path+'peaks',np.int32)
peaks.shape=(-1,2)
peak_starts=np.fromfile(path+'peak_starts',np.int32)
#peak_starts.shape=(-1,2)
pulse_starts=np.fromfile(path+'peak_starts',np.int32)
settings=np.fromfile(path+'settings',np.int32)
cfd_low=np.fromfile(path+'cfdlow',np.int32)
cfd_high=np.fromfile(path+'cfdhigh',np.int32)
slope_xing=np.fromfile(path+'slopethreshxings',np.int32)


In [ ]:
settings[13]

In [ ]:
#create register dict from settings array

class height_type(Enum):
    def from_int(value):
        if value == 0:
            return height_type.peak_height
        elif value == 1:
            return height_type.cfd_high
        elif value == 2:
            return height_type.slope_integral
        else:
            raise AttributeError()
    peak_height=0
    cfd_high=1
    slope_integral=2

class trigger_type(Enum):
    def from_int(value):
        if value == 0:
            return trigger_type.pulse_threshold
        elif value == 1:
            return trigger_type.slope_threshold
        elif value == 2:
            return trigger_type.cfd_low
        else:
            raise AttributeError()
    pulse_threshold=0
    slope_threshold=1
    cfd_low=2

class event_type(Enum):
    def from_int(value):
        if value == 0:
            return event_type.peak
        elif value == 1:
            return event_type.area
        elif value == 2:
            return event_type.pulse
        elif value == 3:
            return event_type.trace
        else:
            raise AttributeError()
    peak=0
    area=1
    pulse=2
    trace=3


def read_registers(settings):
    registers = dict()
    registers['baseline'] = dict()
    registers['baseline']['offset'] = settings[0]
    registers['baseline']['subtraction'] = settings[1]!=0
    registers['baseline']['timeconstant'] = settings[2]
    registers['baseline']['threshold'] = settings[3]
    registers['baseline']['count_threshold'] = settings[4]
    registers['generic'] = dict()
    registers['baseline']['average_order'] = settings[5]
    registers['capture'] = dict()
    registers['capture']['cfd_relative'] = settings[6]!=0
    registers['capture']['constant_fraction'] = settings[7]
    registers['capture']['pulse_threshold'] = settings[8]
    registers['capture']['slope_threshold'] = settings[9]
    registers['capture']['pulse_area_threshold'] = settings[10]
    registers['capture']['height_type'] = height_type.from_int(settings[11])
    registers['capture']['threshold_rel2min'] = settings[12]!=0
    registers['capture']['trigger_type'] = trigger_type.from_int(settings[13])
    registers['capture']['event_type'] = event_type.from_int(settings[14])
    return registers


In [ ]:
%matplotlib notebook
x=np.arange(len(traces['filtered']))
fig=plt.figure()

ax=fig.add_axes([0.15,0.1,0.8,0.8])
mV=1.0/math.pow(2,14)*1000
nS=4


i_trace, =ax.step(x*nS/1000000,(traces['input']+1800)*mV*2,'k',label='raw input')
f_trace, =ax.step(x*nS/1000000,traces['filtered']*mV,'b',label='processed')
fig.suptitle('Digital Signal Processing',fontsize=24)
plt.xlabel('time (ms)',fontsize=18)
plt.ylabel('Voltage (mV)',fontsize=18)
plt.legend(handles=[i_trace,f_trace])

fig.savefig('trace.png')
plt.show()

In [ ]:
%matplotlib notebook
import math

def align_yaxis(ax1, v1, ax2, v2):
    """adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1"""
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    adjust_yaxis(ax2,(y1-y2)/2,v2)
    adjust_yaxis(ax1,(y2-y1)/2,v1)

def adjust_yaxis(ax,ydif,v):
    """shift axis ax by ydiff, maintaining point v at the same location"""
    inv = ax.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, ydif))
    miny, maxy = ax.get_ylim()
    miny, maxy = miny - v, maxy - v
    if -miny>maxy or (-miny==maxy and dy > 0):
        nminy = miny
        nmaxy = miny*(maxy+dy)/(miny+dy)
    else:
        nmaxy = maxy
        nminy = maxy*(miny+dy)/(maxy+dy)
    ax.set_ylim(nminy+v, nmaxy+v)
    
x=np.arange(len(traces['filtered']))
fig=plt.figure()
ax=fig.add_axes([0.1,0.1,0.8,0.8])
plt.ylim(-10,120)
v=math.pow(2,13)/1000
t=x*4
ax.plot([0,t[-1]],[0,0],'k')
raw, = ax.step(t,traces['raw']/2/v,'k',lw=1,label='raw')
processed, = ax.step(t,traces['filtered']/2/v,'r',lw=2,label='processed')
ax.plot(trigger,traces['filtered'][trigger+1]/2,'kx')
ax.plot(peaks[:,0],traces['filtered'][peaks[:,0]+1]/2,'ro')
ax.plot(peak_starts,traces['filtered'][peak_starts+1]/2,'ro')
ax.plot(cfd_high,traces['filtered'][cfd_high+1]/2,'bs')
ax.plot(cfd_low,traces['filtered'][cfd_low+1]/2,'bs')
plt.xlabel('time (ns)',fontsize=18)
plt.ylabel('Voltage (mV)',fontsize=18,color='r')

ax2=ax.twinx()
slope, = ax2.step(t,traces['slope']/256/v/4,'b',lw=2,label='slope')
plt.xlim([1292000,1297500])
plt.ylim(-0.2,1)
align_yaxis(ax,0,ax2,0)
plt.ylabel('slope (mV/ns)',color='b',fontsize=18)
fig.suptitle('Digital signal processing',fontsize=24)
plt.legend(handles=[raw,processed,slope])
fig.savefig('dsp.png')
plt.show()

In [ ]:
%matplotlib notebook

#plot pulse
from mpl_toolkits.axes_grid1 import host_subplot
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

pulse=7
length=2000
pre=100

def align_yaxis(ax1, v1, ax2, v2):
    """adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1"""
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    adjust_yaxis(ax2,(y1-y2)/2,v2)
    adjust_yaxis(ax1,(y2-y1)/2,v1)

def adjust_yaxis(ax,ydif,v):
    """shift axis ax by ydiff, maintaining point v at the same location"""
    inv = ax.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, ydif))
    miny, maxy = ax.get_ylim()
    miny, maxy = miny - v, maxy - v
    if -miny>maxy or (-miny==maxy and dy > 0):
        nminy = miny
        nmaxy = miny*(maxy+dy)/(miny+dy)
    else:
        nmaxy = maxy
        nminy = maxy*(miny+dy)/(maxy+dy)
    ax.set_ylim(nminy+v, nmaxy+v)
    
start=pulse_starts[pulse]-pre
stop=start+length
x=np.arange(0,len(traces['filtered']))
fig=plt.figure()

f_ax = host_subplot(111)
s_ax = f_ax.twinx()

f_ax.set_xlabel("Time (ns)",fontsize=18)
f_ax.set_ylabel("Voltage (mV)",fontsize=18,color='r')
s_ax.set_ylabel("Slope (mv/ns)",fontsize=18,color='b')

mV=1.0/math.pow(2,14)*1000
nS=4
t=x*nS

f_sig = traces['filtered']*mV
s_sig = traces['slope']*mV/nS

f_ax.plot([t[start],t[stop]],[0,0],'k') # zero line
f_trace, = f_ax.step(t[start:stop],f_sig[start:stop],'r',lw=2,label='processed')
f_ax.set_ylim(-10,180)

s_trace, = s_ax.step(t[start:stop],s_sig[start:stop],'b',lw=1,label='slope')
s_ax.set_ylim(-15,50)
#s_threshold = 6
#s_ax.set_yticks(np.concatenate((s_ax.get_yticks(),[s_threshold])))
#labels=[label.get_text() for label in s_ax.get_yticklabels()]
#labels[6]='Threshold'

#s_ax.set_yticklabels(labels)
plt.xlim(t[start],t[stop])
align_yaxis(f_ax,0,s_ax,0)
fig.suptitle('Measurement:First pass',fontsize=24)
#plt.legend(handles=[f_trace,s_trace])

starts_mask = np.logical_and(peak_starts >= start,peak_starts <= stop)
peaks_mask = np.logical_and(peaks[:,0] >= start, peaks[:,0] <= stop)
peak_starts_inwin = peak_starts[starts_mask]

for pstart in peak_starts_inwin:
    peak_start_t=t[pstart]-2
    peak_start_f=f_sig[pstart]
    peak_start_s=s_sig[pstart]

    f_ax.plot(peak_start_t,peak_start_f,'ro')
    s_ax.plot(peak_start_t,peak_start_s,'bo')
    
    f_trans = f_ax.transData
    s_trans = s_ax.transData.inverted()
    trans = s_trans+f_trans
    f_point=f_trans.transform((peak_start_t,peak_start_f))
    s_point=s_trans.transform((f_point[0],f_point[1]))

    s_ax.plot([peak_start_t,peak_start_t],[peak_start_s, s_point[1]],'k')

#for p in peaks[0][peaks_inwin]
peaks_inwin = peaks[peaks_mask,0]
for p in peaks_inwin:
    peak_t=t[p]-2
    peak_f=f_sig[p]
    peak_s=s_sig[p]

    f_ax.plot(peak_t,peak_f,'ro')
    s_ax.plot(peak_t,peak_s,'bo')
    
    f_trans = f_ax.transData
    s_trans = s_ax.transData.inverted()
    trans = s_trans+f_trans
    f_point=f_trans.transform((peak_t,peak_f))
    s_point=s_trans.transform((f_point[0],f_point[1]))

    s_ax.plot([peak_t,peak_t],[peak_s, s_point[1]],'k')

ins_start = peak_starts_inwin[0]-5
ins_stop = peak_starts_inwin[0]+5
f_ins_ax = zoomed_inset_axes(f_ax, 50, loc=1)  # zoom = 6
f_ins_ax.step(t[ins_start:ins_stop],f_sig[ins_start:ins_stop],'r')
f_ins_ax.set_ylim(-0.5,2)
f_ins_ax.set_xlim(ins_start,ins_stop)
f_ins_ax.set_yticks([])
f_ins_ax.set_xticks([])

f_ins_ax.plot([t[ins_start],t[ins_stop]],[0,0],'k') # zero line

#plt.xticks(visible=False)
#plt.yticks(visible=False)
mark_inset(f_ax, f_ins_ax, loc1=2, loc2=4, fc="none", ec="0.5")

s_ins_ax = f_ins_ax.twinx()
s_ins_ax.set_yticks([])
s_ins_ax.set_xticks([])

s_ins_ax.step(t[ins_start:ins_stop],s_sig[ins_start:ins_stop],'b')

s_ins_ax.set_ylim(-1,1)
align_yaxis(f_ins_ax,0,s_ins_ax,0)

pstart = peak_starts_inwin[0]
peak_start_t=t[pstart]-2
peak_start_f=f_sig[pstart]
peak_start_s=s_sig[pstart]

f_ins_ax.plot(peak_start_t,peak_start_f,'ro')
s_ins_ax.plot(peak_start_t,peak_start_s,'bo')
    
f_trans = f_ins_ax.transData
s_trans = s_ins_ax.transData.inverted()
trans = s_trans+f_trans
f_point=f_trans.transform((peak_start_t,peak_start_f))
s_point=s_trans.transform((f_point[0],f_point[1]))

s_ins_ax.plot([peak_start_t,peak_start_t],[peak_start_s, s_point[1]],'k')

s_ax.plot([t[start],t[stop]],[s_threshold,s_threshold],ls='dashed')

fig.savefig('measurement1.png')
plt.draw()
plt.show()

In [ ]:
%matplotlib notebook
from mpl_toolkits.axes_grid1 import host_subplot
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

pulse=7
length=2000
pre=100

def align_yaxis(ax1, v1, ax2, v2):
    """adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1"""
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    adjust_yaxis(ax2,(y1-y2)/2,v2)
    adjust_yaxis(ax1,(y2-y1)/2,v1)

def adjust_yaxis(ax,ydif,v):
    """shift axis ax by ydiff, maintaining point v at the same location"""
    inv = ax.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, ydif))
    miny, maxy = ax.get_ylim()
    miny, maxy = miny - v, maxy - v
    if -miny>maxy or (-miny==maxy and dy > 0):
        nminy = miny
        nmaxy = miny*(maxy+dy)/(miny+dy)
    else:
        nmaxy = maxy
        nminy = maxy*(miny+dy)/(maxy+dy)
    ax.set_ylim(nminy+v, nmaxy+v)
    
start=pulse_starts[pulse]-pre
stop=start+length
x=np.arange(0,len(traces['filtered']))
fig=plt.figure()

f_ax = host_subplot(111)
s_ax = f_ax.twinx()

f_ax.set_xlabel("Time (ns)",fontsize=18)
f_ax.set_ylabel("Voltage (mV)",fontsize=18,color='r')
s_ax.set_ylabel("Slope (mv/ns)",fontsize=18,color='b')

mV=1.0/math.pow(2,14)*1000
nS=4
t=x*nS

f_sig = traces['filtered']*mV
s_sig = traces['slope']*mV/nS

f_ax.plot([t[start],t[stop]],[0,0],'k') # zero line
f_trace, = f_ax.step(t[start:stop],f_sig[start:stop],'r',lw=2,label='processed')
f_ax.set_ylim(-10,180)

s_trace, = s_ax.step(t[start:stop],s_sig[start:stop],'b',lw=1,label='slope')
s_ax.set_ylim(-15,50)
#s_threshold = 6
#s_ax.set_yticks(np.concatenate((s_ax.get_yticks(),[s_threshold])))
#labels=[label.get_text() for label in s_ax.get_yticklabels()]
#labels[6]='Threshold'

#s_ax.set_yticklabels(labels)
plt.xlim(t[start],t[stop])
align_yaxis(f_ax,0,s_ax,0)
fig.suptitle('Measurement:Second pass',fontsize=24)
#plt.legend(handles=[f_trace,s_trace])

starts_mask = np.logical_and(peak_starts >= start,peak_starts <= stop)
peaks_mask = np.logical_and(peaks[:,0] >= start, peaks[:,0] <= stop)
peak_starts_inwin = peak_starts[starts_mask]


#for pstart in peak_starts_inwin:
#    peak_start_t=t[pstart]-2
#    peak_start_f=f_sig[pstart]
#    peak_start_s=s_sig[pstart]
#
#    f_ax.plot(peak_start_t,peak_start_f,'ro')
#    s_ax.plot(peak_start_t,peak_start_s,'bo')
    
#    f_trans = f_ax.transData
#    s_trans = s_ax.transData.inverted()
#    trans = s_trans+f_trans
#    f_point=f_trans.transform((peak_start_t,peak_start_f))
#    s_point=s_trans.transform((f_point[0],f_point[1]))

#    s_ax.plot([peak_start_t,peak_start_t],[peak_start_s, s_point[1]],'k')

#for p in peaks[0][peaks_inwin]
peaks_inwin = peaks[peaks_mask,0]
x_s=peak_starts_inwin
x_p=peaks_inwin
y_s=f_sig[x_s]
y_p=f_sig[x_p]

f_ax.plot(t[peak_starts_inwin],f_sig[peak_starts_inwin],'or')
f_ax.plot(t[peaks_inwin],f_sig[peaks_inwin],'or')


f_ax.plot([t[x_s[0]],t[x_p[0]]],[y_s[0],y_s[0]],'k')
f_ax.plot([t[x_p[0]],t[x_p[0]]],[y_s[0],y_p[0]],'k')

f_ax.plot([t[x_s[1]],t[x_p[1]]],[y_s[1],y_s[1]],'k')
f_ax.plot([t[x_p[1]],t[x_p[1]]],[y_s[1],y_p[1]],'k')


#for p in peaks_inwin:
#    peak_t=t[p]-2
#    peak_f=f_sig[p]
#    peak_s=s_sig[p]

#    f_ax.plot(peak_t,peak_f,'ro')
#    s_ax.plot(peak_t,peak_s,'bo')
    
#    f_trans = f_ax.transData
#    s_trans = s_ax.transata.inverted()
#    trans = s_trans+f_trans
#    f_point=f_trans.transform((peak_t,peak_f))
#    s_point=s_trans.transform((f_point[0],f_point[1]))

#    s_ax.plot([peak_t,peak_t],[peak_s, s_point[1]],'k')

ins_start = peak_starts_inwin[1]-300
ins_stop = peak_starts_inwin[1]+500
f_ins_ax = zoomed_inset_axes(f_ax, 1.5, loc=1)  # zoom = 6
f_ins_ax.step(t[ins_start:ins_stop],f_sig[ins_start:ins_stop],'r')
f_ins_ax.set_ylim(60,180)
f_ins_ax.set_xlim(t[ins_start],t[ins_stop])
f_ins_ax.set_yticks([])
f_ins_ax.set_xticks([])

low_mask = np.logical_and(cfd_low >= start, cfd_low <= stop)
low_inwin = cfd_low[low_mask]
high_mask = np.logical_and(cfd_high >= start, cfd_high <= stop)
high_inwin = cfd_high[high_mask]
thresh_mask = np.logical_and(slope_xing >= start, slope_xing <= stop)
thresh_inwin = slope_xing[thresh_mask]

f_ins_ax.plot(t[peak_starts_inwin],f_sig[peak_starts_inwin],'or')
f_ins_ax.plot(t[peaks_inwin],f_sig[peaks_inwin],'or')

f_ins_ax.plot(t[low_inwin],f_sig[low_inwin],'ok')
f_ins_ax.plot(t[high_inwin],f_sig[high_inwin],'ok')
f_ax.plot(t[low_inwin],f_sig[low_inwin],'ok')
f_ax.plot(t[high_inwin],f_sig[high_inwin],'ok')
#f_ax.plot(t[thresh_inwin],f_sig[thresh_inwin],'ow')
f_ins_ax.plot(t[thresh_inwin],f_sig[thresh_inwin],'ow')


f_ins_ax.plot([t[x_s[0]],t[x_p[0]]],[y_s[0],y_s[0]],'k')
f_ins_ax.plot([t[x_p[0]],t[x_p[0]]],[y_s[0],y_p[0]],'k')

f_ins_ax.plot([t[x_s[1]],t[x_p[1]]],[y_s[1],y_s[1]],'k')
f_ins_ax.plot([t[x_p[1]],t[x_p[1]]],[y_s[1],y_p[1]],'k')

#f_ins_ax.plot([t[ins_start],t[ins_stop]],[0,0],'k') # zero line

#plt.xticks(visible=False)
#plt.yticks(visible=False)
#mark_inset(f_ax, f_ins_ax, loc1=2, loc2=4, fc="none", ec="0.5")

pstart = peak_starts_inwin[1]
peak_start_t=t[pstart]-2
peak_start_f=f_sig[pstart]
peak_start_s=s_sig[pstart]

f_ins_ax.plot(peak_start_t,peak_start_f,'ro')


f_trans = f_ins_ax.transData
s_trans = s_ins_ax.transData.inverted()
trans = s_trans+f_trans
f_point=f_trans.transform((peak_start_t,peak_start_f))
s_point=s_trans.transform((f_point[0],f_point[1]))

s_ins_ax.plot([peak_start_t,peak_start_t],[peak_start_s, s_point[1]],'k')

s_ax.plot([t[start],t[stop]],[s_threshold,s_threshold],ls='dashed')

fig.savefig('measurement2.png')
plt.draw()
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

fig=plt.figure()
fig.suptitle('Multi Channel Analyser',fontsize=24)

h_ax = host_subplot(111)

h_ins_ax = inset_axes(h_ax, width="80%",height=1., loc=1)  # zoom = 6
h_ins_ax.set_xlim(-25,160)
h_ins_ax.set_ylim(0,0.002)
h_ax.hist((traces['input']+200)*2*mV,math.pow(2,14),normed=1,facecolor='k',edgecolor='k')
h_ax.hist(f_sig,math.pow(2,14),normed=1,facecolor='r',edgecolor='r')
h_ins_ax.hist(f_sig,math.pow(2,14),normed=1,facecolor='r',edgecolor='r')
h_ins_ax.set_xticks([])
h_ins_ax.set_yticks([])
h_ax.set_xlim(-30,100)
h_ax.set_ylim(0,0.8)

mark_inset(h_ax, h_ins_ax, loc1=2, loc2=4, fc="none", ec="0.5")

h_ax.set_xlabel('Voltage (mV)')
h_ax.set_ylabel('Probability')
fig.savefig('hist.png')
plt.show()

In [ ]:
peak_dt=np.dtype([('time',np.uint16),('flags',np.uint16),('minima',np.uint16),('height',np.uint16)])

In [ ]:
area_dt=np.dtype([('time',np.uint16),('flags',np.uint16),('area',np.uint32)])

In [ ]:
pulse_peak_dt=np.dtype([('trigger',np.uint16),('rise',np.uint16),('minima',np.uint16),('height',np.uint16)])
pulse_header_dt=np.dtype([('time',np.uint16),('flags',np.uint16),('res1',np.uint16),('size',np.uint16),
                    ('length',np.uint16),('res2',np.uint16),('area',np.int32)])

In [ ]:
pulse2_dt=np.dtype([('time',np.uint16),('flags',np.uint16),('res1',np.uint16),('size',np.uint16),
                    ('length',np.uint16),('res2',np.uint16),('area',np.int32),
                    ('trigger0',np.uint16),('rise0',np.uint16),('minima0',np.uint16),('height0',np.uint16),
                    ('trigger1',np.uint16),('rise1',np.uint16),('minima1',np.uint16),('height1',np.uint16)])

In [ ]:
pulse2_subarray_dt=np.dtype([('header',pulse_header_dt,('peaks', (pulse_peak_dt,(1,2)) ))])

In [ ]:
eventstream=np.fromfile(path+'eventstream',peak_dt)

In [ ]:
eventstream=np.fromfile(path+'eventstream',area_dt)

In [ ]:
eventstream=np.fromfile(path+'eventstream',pulse2_dt)

In [ ]:
eventstream

In [ ]:
flags=eventstream['flags']
print(np.binary_repr(flags[0],16))

In [ ]:
print("flags:{:016b}\nsize:{}\nlength:{}\narea:{}\ntrigger:{} rise:{} minima:{} height:{}\ntrigger:{} rise:{} minima:{} height:{}\n"  
      .format(eventstream['flags'][0],eventstream['size'][0],eventstream['length'][0],eventstream['area'][0],
        eventstream['trigger0'][0],eventstream['rise0'][0],eventstream['minima0'][0],eventstream['height0'][0],
        eventstream['trigger1'][0],eventstream['rise1'][0],eventstream['minima1'][0],eventstream['height1'][0]))

In [ ]:
print(np.binary_repr(flags[1],16))

In [ ]:
rel_to_min=np.right_shift(np.bitwise_and(flags,0x8000),15).astype(np.bool)
channel=np.right_shift(np.bitwise_and(flags,0x0F00),8).astype(np.uint8)
threshold_rel2min=np.right_shift(np.bitwise_and(flags,0x4000),14).astype(np.bool)
peak_count=np.right_shift(np.bitwise_and(flags,0x00F0),4).astype(np.uint8)
tick=np.bitwise_and(flags,0x0001).astype(bool)
area=np.right_shift(np.bitwise_and(flags,0x0002),1).astype(np.bool)
trace=np.right_shift(np.bitwise_and(flags,0x0004),2).astype(np.bool)
fixed=np.right_shift(np.bitwise_and(flags,0x0008),3).astype(np.bool)
event_type=np.right_shift(np.bitwise_and(flags,0x3000),12).astype(np.uint8)

In [ ]:
flags

In [ ]:
eventstream